<a href="https://colab.research.google.com/github/RandamD/House_price_prediction/blob/main/StudentPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from flask import Flask, request, jsonify

from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,confusion_matrix,classification_report,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/": {"origins": ""}})

# # Load the model and scaler (make sure to run this after saving your model and scaler)
# model = pickle.load(open('lin_reg_model.pkl', 'rb'))
# scaler = pickle.load(open('scaler.pkl', 'rb'))
# encoder = pickle.load(open('encoder.pkl', 'rb'))


# Set up your authtoken
ngrok.set_auth_token("2n48JoTRdodKQP87MZDzWQmOdzd_38ADmTYqLNQQoKJgxQFe3")  # Replace YOUR_NGROK_AUTHTOKEN with your actual authtoken

# Set up ngrok tunnel to expose the app to the internet
public_url = ngrok.connect(5000)
print("Public URL:", public_url)



# Data loading
df = pd.read_csv('student_performance.csv')
df = df.drop(['StudentID', 'Name'], axis=1)

# Encode categorical variables
lc = LabelEncoder()
df['Gender'] = lc.fit_transform(df['Gender'])
df['ParentalSupport'] = lc.fit_transform(df['ParentalSupport'])

# Define features and target
features = ['Gender', 'AttendanceRate', 'StudyHoursPerWeek', 'PreviousGrade', 'ExtracurricularActivities', 'ParentalSupport']
x = df[features]
y = df['FinalGrade']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[['StudyHoursPerWeek', 'AttendanceRate']] = scaler.fit_transform(X_train[['StudyHoursPerWeek', 'AttendanceRate']])
X_test_scaled[['StudyHoursPerWeek', 'AttendanceRate']] = scaler.transform(X_test[['StudyHoursPerWeek', 'AttendanceRate']])

# Train the model
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# Save the model, scaler, and encoder
pickle.dump(linreg, open('/content/lin_reg_model.pkl', 'wb'))
pickle.dump(scaler, open('/content/scaler.pkl', 'wb'))
pickle.dump(lc, open('/content/encoder.pkl', 'wb'))

# Load the model and scaler
model = pickle.load(open('/content/lin_reg_model.pkl', 'rb'))
scaler = pickle.load(open('/content/scaler.pkl', 'rb'))
encoder = pickle.load(open('/content/encoder.pkl', 'rb'))



@app.route('/data', methods=['POST'])
def get_data():
    data = request.json
    print("Received data:", data)

    # Create a DataFrame from the incoming JSON data
    df_input = pd.DataFrame([data])

    # Encode categorical variables
    df_input['Gender'] = df_input['Gender'].map({'Male': 1, 'Female': 0})
    df_input['ParentalSupport'] = df_input['ParentalSupport'].map({'Low': 0, 'Medium': 1, 'High': 2})
    df_input.fillna(0, inplace=True)

    # Ensure DataFrame has correct columns and order
    df_input = df_input[features]

    # Scale numerical features (only scale the features that were scaled during training)
    df_input[['StudyHoursPerWeek', 'AttendanceRate']] = scaler.transform(df_input[['StudyHoursPerWeek', 'AttendanceRate']])

    # Make predictions
    prediction = model.predict(df_input)

    response = {"message": "Prediction successful", "predicted_final_grade": prediction[0]}
    return jsonify(response)

if __name__ == '__main__':
    app.run(port=5000)

ModuleNotFoundError: No module named 'pyngrok'

In [ ]:
!pip install Flask-Cors

In [ ]:
!ngrok kill

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,confusion_matrix,classification_report,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
!pip install pyngrok

In [ ]:
df= pd.read_csv('student_performance.csv')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df['Gender'].value_counts()

In [ ]:
df=df.drop(['StudentID','Name'],axis=1)

In [ ]:
df

In [ ]:
lc=LabelEncoder()
df['Gender']=lc.fit_transform(df['Gender'])
df['ParentalSupport']=lc.fit_transform(df['ParentalSupport'])
df

In [ ]:
features = ['Gender', 'AttendanceRate', 'StudyHoursPerWeek', 'PreviousGrade', 'ExtracurricularActivities', 'ParentalSupport']
x=df[features]
y=df['FinalGrade']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape
scaler=StandardScaler()

In [ ]:
X_train.shape,X_test.shape

In [ ]:
scaler=StandardScaler()

In [ ]:
X_train_scaled=X_train
X_test_scaled = X_test

In [ ]:
X_train_scaled[['StudyHoursPerWeek','AttendanceRate']]=scaler.fit_transform(X_train[['StudyHoursPerWeek','AttendanceRate']])
X_test_scaled[['StudyHoursPerWeek','AttendanceRate']]=scaler.transform(X_test[['StudyHoursPerWeek','AttendanceRate']])

In [ ]:
X_train_scaled

In [ ]:
correlation_matrix  = x.corrwith(y, axis=0)
correlation_matrix=correlation_matrix.sort_values()
print(correlation_matrix)

In [ ]:
correlation_df=pd.DataFrame(correlation_matrix,columns=['correlation'])
correlation_df

In [ ]:


numeric_df = df.select_dtypes(include=np.number)

plt.figure(figsize=(10, 10)) # now plt is defined and can be used to access functions like figure
plt.title('Relations among Features and FinalGrades')
# Use the numeric DataFrame for correlation calculation
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
def correlation(dataset, threshold,target):
  col_corr=set()
  corr_matrix=dataset.corrwith(target,axis=0)
  for i in range(len(corr_matrix)):
    if abs(corr_matrix.iloc[i])>threshold:
      colname=dataset.columns[i]
      col_corr.add(colname+': '+str(abs(corr_matrix.iloc[i])))
  return col_corr

In [ ]:
col_corr=correlation(x,threshold=0.9,target=y)
col_corr

In [ ]:
linreg=LinearRegression()
linreg.fit(X_train_scaled,y_train)
y_pred=linreg.predict(X_test_scaled)

y_pred

In [ ]:
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
mse=mean_squared_error(y_test,y_pred)
rmse=np.sqrt(mse)
print("Mean Absolute Error : ",mae)
print("Mean Squared Error : ",mse)
print("Root Mean Squared Error : ",rmse)
print("R2 Score : ",score)

In [ ]:
plt.scatter(y_test,y_pred)

In [ ]:


pickle.dump(linreg, open('lin_reg_mode.pkl', 'wb'))

In [ ]:
model=pickle.load(open('lin_reg_mode.pkl','rb'))

In [ ]:
pickle.dump(scaler,open('scaler.pkl','wb'))

In [ ]:
pickle.dump(scaler,open('encoder.pkl','wb'))

In [ ]:
model.predict(X_test_scaled)

In [ ]:
y_test,y_pred

In [ ]:
kfold=KFold(n_splits=5,shuffle=True,random_state=42)
scores=cross_val_score(linreg,x,y,cv=kfold)
print(scores)

In [4]:
!pip install pyngrok

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5